In [1]:
from transformers import pipeline
import torch

pipe = pipeline("summarization", model="t5-small",device=0)

b:\LHL\LHL-LLM\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
b:\LHL\LHL-LLM\.conda\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ElenaFall\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: http

In [ ]:
import pandas as pd

df_train = pd.read_pickle('pickles\df_train_prepro.pickle')
df_test = pd.read_pickle('pickles\df_train_token_vec.pickle')

In [7]:
from datasets import Dataset, DatasetDict
new_train = Dataset.from_pandas(df_train[['document','summary']])
new_test = Dataset.from_pandas(df_train_token[['document','summary']])

new_ds = DatasetDict({
    'train': new_train,
    'test': new_test})
new_ds

KeyError: "None of [Index(['document', 'summary'], dtype='object')] are in the [columns]"

In [ ]:
data = new_ds['test'][0:1000]
preds = pipe(data,truncation=True, batch_size=2)

ValueError:  `args[0]`:                                                 document  \
0      national archives yes it’s time folks it’s fir...   
1      los angeles ap — first interview since nba ban...   
2      gaithersburg md ap — small private jet crashed...   
3      tucker carlson exposes sexism twitter updated ...   
4      man accused removing another mans testicle mee...   
...                                                  ...   
44967  670000 copies pearls’ selfpublished book circu...   
44968  seeking costconscious consumers gravitated tow...   
44969  click email friend opens new window click shar...   
44970  barrington ri ap — women clad yoga pants plan ...   
44971  based ‘real’ story hit john travolta film satu...   

                                                 summary  
0      – The unemployment rate dropped to 8.2% last m...  
1      – Shelly Sterling plans "eventually" to divorc...  
2      – A twin-engine Embraer jet that the FAA descr...  
3      – Tucker Car

In [4]:
from tqdm import tqdm
batch_size = 2  # Adjust this value based on your GPU memory
all_preds = []
for i in tqdm(range(0, len(df_train), batch_size)):
    batch = df_train['document'].iloc[i:i+batch_size].tolist()
    batch_preds = pipe(batch, max_length=150, min_length=40, do_sample=False)
    all_preds.extend([pred['summary_text'] for pred in batch_preds])

    # Save progress every 10 batches
    if i % (batch_size * 10) == 0 and i > 0:
        temp_df = df_train.iloc[:len(all_preds)].copy()
        temp_df['generated_summary'] = all_preds
        temp_df.to_pickle(f'pickles/df_train_with_summaries_temp_{i}.pickle')

df_train['generated_summary'] = all_preds

df_train.to_pickle('pickles/df_train_with_summaries.pickle')


  0%|          | 59/22486 [09:46<61:54:25,  9.94s/it] 


OutOfMemoryError: CUDA out of memory. Tried to allocate 36.14 GiB. GPU 0 has a total capacity of 6.00 GiB of which 3.65 GiB is free. Of the allocated memory 1.15 GiB is allocated by PyTorch, and 262.80 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# preds = pipe(list(df_train['document']), batch_size=1)

Your max_length is set to 200, but your input_length is only 136. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=68)


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.35 GiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 12.88 GiB is allocated by PyTorch, and 3.60 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)